In [1]:
# Pkg.add("Interact")
# Pkg.add("DataStructures")

using Tokamak
using DataStructures
using Interact


Use "abstract type Widget end" instead.

Use "abstract type InputWidget{T}<:Widget end" instead.

Use "abstract type WidgetMod<:Widget end" instead.

Use "abstract type Container<:Widget end" instead.


In [10]:
function parse_tokamak_data(solved_case)
    cur_table_data = OrderedDict(
        "eta_CD" => @sprintf("%.3g", solved_case["eta_CD"]),
        "B_0" => @sprintf("%.4g", solved_case["B_0"]),
        "P_F" => @sprintf("%.4g", solved_case["P_F"]),
        "I_M" => @sprintf("%.3g", solved_case["I_M"]),
        "R_0" => @sprintf("%.3g", solved_case["R_0"]),
        " a " => @sprintf("%.3g", solved_case["a"]),
        "q_K" => @sprintf("%.3g", solved_case["q_star"]),
        "T_i" => @sprintf("%.3g", solved_case["T_k"]),
        "n_e" => @sprintf("%.3g", solved_case["n_bar"]),
        "f_CD" => @sprintf("%.3g", solved_case["f_CD"]),
        "f_B" => @sprintf("%.3g", solved_case["f_B"]),
        "P_H" => @sprintf("%.3g", solved_case["P_H"]),
        "beta" => @sprintf("%.4g", solved_case["beta"] * Tokamak.max_beta_N),
        "P_W" => @sprintf("%.4g", solved_case["P_W"] * Tokamak.max_P_W),
        "P_alpha" => @sprintf("%.4g", solved_case["P_alpha"]),
        "P_BR" => @sprintf("%.4g", solved_case["P_BR"]),
        "q_P" => @sprintf("%.4g", solved_case["q_P"]),
        "constraint" => solved_case["constraint"],
        "rho_j" => @sprintf("%.4g", solved_case["rho_j"])
    ) 
    
    cur_table_data
end

parse_tokamak_data (generic function with 1 method)

In [11]:
function act_1_data()
    cur_table_data = OrderedDict()
    
    cur_table_data["main"] = OrderedDict(
        "eta_CD" => 0.4,
        "B_0" => 6.0,
        "P_F" => 1813,
        "I_M" => 10.95,
        "R_0" => 6.25,
        " a " => 1.56,
        "q_K" => 4.5,
        "T_i" => 20.6,
        "n_e" => 1.3,
        "f_CD" => 0.09,
        "f_B" => 0.91,
        "P_H" => 42.7,
        "beta" => 0.056,
        "P_W" => 2.45,
        "P_alpha" => 363,
        "P_BR" => 56.3,
        "q_P" => 13.7,
        "constraint" => "-",
        "rho_j" => "-"
    ) 

    cur_table_data
end

act_1_data (generic function with 1 method)

In [12]:
function act_2_data()
    cur_table_data = OrderedDict()
    
    cur_table_data["main"] = OrderedDict(
        "eta_CD" => 0.4,
        "B_0" => 8.75,
        "P_F" => 2637,
        "I_M" => 13.98,
        "R_0" => 9.75,
        " a " => 2.44,
        "q_K" => 8.0,
        "T_i" => 17.8,
        "n_e" => 0.86,
        "f_CD" => 0.23,
        "f_B" => 0.77,
        "P_H" => 105.5,
        "beta" => 0.026,
        "P_W" => 1.46,
        "P_alpha" => 528,
        "P_BR" => 96.5,
        "q_P" => 10.0,
        "constraint" => "-",
        "rho_j" => "-"
    ) 

    cur_table_data
end

act_2_data (generic function with 1 method)

In [13]:
function make_output_table(solved_cases, design, column)
    
    tokamak_jl_data = OrderedDict()
    
    for (cur_key, cur_val) in solved_cases
        tokamak_jl_data[cur_key] = parse_tokamak_data(cur_val)
    end
        
    if design == "Act 1"
        paper_data = act_1_data()
    elseif design == "Act 2"
        paper_data = act_2_data()
    else
        error("Invalid design for paper selection")
    end
    
    header_row = ["Variable", "Paper"]
 
    for cur_key in keys(tokamak_jl_data)
        push!(header_row, cur_key)
    end

    num_keys = length(keys(paper_data[column]))
    
    mapped_data = [ [] for ii in 1:num_keys ]
    
    for (cur_index, cur_key) in enumerate(keys(paper_data[column]))
        push!(mapped_data[cur_index], cur_key)
        
        push!(mapped_data[cur_index], paper_data[column][cur_key])
        
        for cur_val in values(tokamak_jl_data)
            push!(mapped_data[cur_index], cur_val[cur_key])
        end
    end
    
    cur_table = vcat( (reshape(x, (1, :)) for x in mapped_data)... )

    Tokamak.HTMLElements.table(cur_table, header_row=header_row, title="$design Input")    
end

make_output_table (generic function with 1 method)

In [14]:
B_0_dict = OrderedDict(
    "act_1" => 6.0,
    "act_2" => 8.75
)

DataStructures.OrderedDict{String,Float64} with 2 entries:
  "act_1" => 6.0
  "act_2" => 8.75

In [22]:
param_dict = Dict(
    "nu_n" => linspace(round(0.41-0.25,3),round(0.41+0.25,3),21),
    "nu_T" => linspace(round(1.15-0.25,3),round(1.15+0.25,3),11),
    "f_DT" => 0.5:0.05:1.0,
    "BR_scaling" => linspace(-0.5,2.5,13),
    "enable_eta_CD_derive" => [false, true],
    "limit" => ["auto", "beta", "wall"],
    "default_eta_CD" => linspace(0.1,0.3,9),
    "rho_m" => linspace(0.3,0.9,13),
    "Z_eff" => linspace(round(2.12-0.72,2),round(2.12+0.72),9),
    "wave_theta" => 0:30:360,
    "Q" => 15:2.5:45
)

other_keys = ["none"]
append!(other_keys, [ cur_key for cur_key in keys(param_dict)] )

default_vals = Dict()

for (cur_key, cur_val) in param_dict
    if cur_key == "wave_theta"
        default_vals[cur_key] = 60
    elseif isa(cur_val, Array)
        default_vals[cur_key] = first(cur_val)
    else
        default_vals[cur_key] = mean(cur_val)
    end
end

default_vals

Dict{Any,Any} with 11 entries:
  "limit"                => "auto"
  "Q"                    => 30.0
  "nu_n"                 => 0.41
  "f_DT"                 => 0.75
  "Z_eff"                => 2.2
  "nu_T"                 => 1.15
  "default_eta_CD"       => 0.2
  "BR_scaling"           => 1.0
  "enable_eta_CD_derive" => false
  "rho_m"                => 0.6
  "wave_theta"           => 60

In [26]:
@manipulate for design=["act_1", "act_2"], column=["main"], limit=param_dict["limit"], 
    eta_CD=param_dict["enable_eta_CD_derive"], Q in slider(param_dict["Q"], value=42.5, label="Q"), 
    f_DT=param_dict["f_DT"], BR=param_dict["BR_scaling"], nu_n=slider(linspace(0.27,0.31,9), value=0.27, label="nu_n"), nu_T=param_dict["nu_T"], 
    rho_m=param_dict["rho_m"], kappa in slider(linspace(1.8,2.2,9), value=1.8, label="kappa"), 
    delta in slider(linspace(0.575,0.63,3), value=0.575, label="delta"), Z_eff=slider(param_dict["Z_eff"], value=2.11, label="Z_eff"), 
    CD_guess=param_dict["default_eta_CD"], wave_theta in slider(param_dict["wave_theta"], 
        value=default_vals["wave_theta"], label="theta"), other_key=other_keys, other_val="", 
    ready=[true, false]
    
    if !ready
        return
    end    
    
    solved_cases = OrderedDict()
    
    if other_key == "none" || other_val == ""
        key_list = ["Tokamak.jl"]
    else
        if other_key == "f_DT"
            first_val = f_DT
        elseif other_key == "BR_scaling"
            first_val = BR
        elseif other_key == "rho_m"
            first_val = rho_m
        elseif other_key == "enable_eta_CD_derive"
            first_val = eta_CD
        elseif other_key == "nu_n"
            first_val = nu_n
        elseif other_key == "nu_T"
            first_val = nu_T            
        elseif other_key == "limit"
            first_val = limit
        elseif other_key == "Z_eff"
            first_val = Z_eff            
        elseif other_key == "Q"
            first_val = Q                        
        elseif other_key == "wave_theta"
            first_val = wave_theta                        
        end
        
        key_list = ["$other_key = $first_val"]
        
        if other_val == ""
            other_val = default_vals[other_key]
        end
        
        push!(key_list, "$other_key = $other_val")        
    end
    
    Tokamak.load_input("defaults.jl", true)
    Tokamak.load_input("$(design).jl", true)  
    
    cur_max_beta_N = Tokamak.max_beta_N
    cur_max_P_W = Tokamak.max_P_W

    for cur_key in key_list
        Tokamak.load_input("defaults.jl", true)
        Tokamak.load_input("$(design).jl", true)

        Tokamak.load_input(" enable_eta_CD_derive = $eta_CD ")

        Tokamak.load_input(" BR_scaling = $BR ")

        Tokamak.load_input(" nu_n = $nu_n ")
        Tokamak.load_input(" nu_T = $nu_T ")

        Tokamak.load_input(" kappa = $kappa ")
        Tokamak.load_input(" delta = $delta ")
        
        Tokamak.load_input(" rho_m = $rho_m ")
        Tokamak.load_input(" Z_eff = $Z_eff ")
        
        Tokamak.load_input(" default_eta_CD = $CD_guess ")
        
        Tokamak.load_input(" wave_theta = $wave_theta ")
        
        Tokamak.load_input(" Q = $Q ")
        
        if other_val != "" && cur_key != "Tokamak.jl"
            if other_key != "f_DT" && other_key != "limit"
                Tokamak.load_input(cur_key)
            end
        end
        
        cur_f_DT = f_DT
        cur_limit = limit
     
        if other_val != "" && cur_key != "Tokamak.jl"
            if other_key == "f_DT"
                cur_f_DT = parse(split(cur_key,"=")[2])
            elseif other_key == "limit"
                cur_limit = strip(split(cur_key,"=")[2])
            end    
        end
        
        if design == "act_2" && isapprox(cur_f_DT, 0.75)
            cur_f_DT = 0.74
        end

        Tokamak.load_input(" f_DT = $cur_f_DT ")
        
        if cur_limit == "beta"
            Tokamak.load_input(" max_P_W = $(1000 * cur_max_P_W) ")
        elseif cur_limit == "wall"
            Tokamak.load_input(" max_beta_N = $(1000 * cur_max_beta_N) ")
        end

        found_data = Tokamak.sweep_B_0([B_0_dict[design]])

        filter!((k,v) -> !isa(v, OrderedDict), found_data)

        for (cur_sub_key, cur_sub_val) in found_data
            found_data[cur_sub_key] = first(cur_sub_val)
        end

        Tokamak.load_input(" max_P_W = $(cur_max_P_W) ")
        Tokamak.load_input(" max_beta_N = $(cur_max_beta_N) ")        
        
        solved_cases[cur_key] = Tokamak.analyze_solved_case(found_data)
    end
    
    table_html = make_output_table(solved_cases, replace(ucfirst(design), "_", " "), column)    
    
    IJulia.clear_output(true)
    display("text/html", table_html)    
end

return

Variable,Paper,Tokamak.jl
eta_CD,0.4,NaN
B_0,6.0,NaN
P_F,1813,NaN
I_M,10.95,NaN
R_0,6.25,NaN
a,1.56,NaN
q_K,4.5,NaN
T_i,20.6,NaN
n_e,1.3,NaN
f_CD,0.09,NaN


In [28]:
Tokamak.max_beta_N

0.056

In [29]:
Tokamak.f_DT

0.75

In [30]:
Tokamak.kappa

2.0

In [31]:
Tokamak.delta

0.575